In [16]:
!pip install --upgrade pip
!pip install --upgrade pypdf langchain-text-splitters langchain-community langgraph faiss-cpu langchain-openai python-dotenv rapidocr-onnxruntime langsmith
!pip install -qU "langchain[openai]"

  Using cached pypdf-5.3.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached langchain_community-0.3.17-py3-none-any.whl.metadata (2.4 kB)
  Using cached langgraph-0.2.73-py3-none-any.whl.metadata (17 kB)
  Using cached faiss_cpu-1.10.0-cp313-cp313-macosx_10_14_x86_64.whl.metadata (4.4 kB)
  Using cached langchain_openai-0.3.6-py3-none-any.whl.metadata (2.3 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached rapidocr_onnxruntime-1.2.3-py3-none-any.whl.metadata (4.8 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached PyYAML-6.0.2-cp313-cp313-macosx_10_13_x86_64.whl.metadata (2.1 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.7.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached langgraph_checkpoint-2.0.15-py3-none-any.whl.metad

In [17]:
import os
from dotenv import load_dotenv
from langchain import hub
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain.chat_models import init_chat_model
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [18]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGSMITH_TRACING = os.getenv("LANGSMITH_TRACING", "false")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY", "not provided")

In [19]:
def load_pdfs_recursively(directory):
    all_pages = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(".pdf"):
                pdf_path = os.path.join(root, file)
                loader = PyPDFLoader(pdf_path, extract_images=True)
                pages = loader.load_and_split()
                all_pages.extend(pages)
    return all_pages

pdf_directory = "pdfs"  # path to dir with PDFs

docs = load_pdfs_recursively(pdf_directory)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = FAISS.from_documents(docs, embeddings)

_ = vector_store.add_documents(documents=all_splits)    # index chunks

print("index created!")

index created!


In [20]:
# Invoke LLM
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [21]:
question = "What should I highlight on an Academic Resume?"

In [22]:
response = graph.invoke({
    "question": question
})

print(response["answer"])

On an academic resume, you should highlight your academic and administrative experience, including committee work and any positions held in academic organizations. Include a section for publications and conferences, detailing peer-reviewed and non-peer-reviewed works, as well as ongoing projects. Additionally, emphasize professional development, relevant professional affiliations, and any honors and awards related to your discipline.
